## Dependencies

In [7]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  !pip install fuzzywuzzy[speedup]
else:
  print("No running in colab")

No running in colab


In [8]:
import pandas as pd
import ast
import itertools
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from fuzzywuzzy import process
from time import process_time
import pickle
print('all imported')

all imported


##DataFrame


In [9]:
df_track = pd.read_csv('https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data.csv')

In [10]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [11]:
df_track['artists']

0         ['Sergei Rachmaninoff', 'James Levine', 'Berli...
1                                            ['Dennis Day']
2         ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...
3                                          ['Frank Parker']
4                                            ['Phil Regan']
                                ...                        
170648    ['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...
170649                                         ['Ashnikko']
170650                                          ['MAMAMOO']
170651                                           ['Eminem']
170652                                ['KEVVO', 'J Balvin']
Name: artists, Length: 170653, dtype: object

##Spotify df



In [12]:
#converting string to a list using ast.literal_eval

In [13]:
df_track.artists = df_track.artists.apply(ast.literal_eval)
# returning a list store in a string, str('[a,g,c]') --> list [a,g,c]
type(df_track.artists[0])

list

In [14]:
df_track.artists[0]

['Sergei Rachmaninoff', 'James Levine', 'Berliner Philharmoniker']

In [15]:
# " ,".join(df_track.artists[0]) 
#testing joining a list to a single string separated by delimiter

In [16]:
def lst_join(list_artists):
  """transform from list to string
  to get rid of the [] of the list"""
  Jointed_list = ", ".join(list_artists)
  return Jointed_list

df_track.artists = df_track.artists.apply(lst_join)

In [17]:
df_track[['artists','id']].head()

,artists,id
0,"Sergei Rachmaninoff, James Levine, Berliner Ph...",4BJqT0PrAfrxzMOxytFOIz
1,Dennis Day,7xPhfUan2yNtyFG0cUWkt8
2,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,1o6I8BglA6ylDMrIELygv1
3,Frank Parker,3ftBPsC5vPBKxYSee08FDH
4,Phil Regan,4d6HGyGT8e121BsdKmw9v6


In [18]:
list(df_track.columns.values)

['valence',
 'year',
 'acousticness',
 'artists',
 'danceability',
 'duration_ms',
 'energy',
 'explicit',
 'id',
 'instrumentalness',
 'key',
 'liveness',
 'loudness',
 'mode',
 'name',
 'popularity',
 'release_date',
 'speechiness',
 'tempo']

## Normalize, Standarize, Model


In [19]:
f_labels = ['valence', 'year', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'explicit', 'instrumentalness',
       'key', 'liveness', 'loudness', 'mode','popularity', 'speechiness', 'tempo']
#selected columns to pass to normalize and pass as a Matrix

In [20]:
scaler = MinMaxScaler()

In [21]:
spotify = df_track.copy()

In [22]:
spotify[f_labels] = scaler.fit_transform(spotify[f_labels])

In [23]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)

In [24]:
spotify_matrix = spotify[f_labels].to_numpy()

In [25]:
model_knn.fit(spotify_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [26]:
spotify_matrix[0]

array([0.0594    , 0.        , 0.98594378, 0.28238866, 0.15311207,
       0.211     , 0.        , 0.878     , 0.90909091, 0.665     ,
       0.62491582, 1.        , 0.04      , 0.03773196, 0.3324504 ])

In [27]:
t1_start = process_time()  
track_search = process.extractOne('Clancy Lowered the Boom', spotify['name'])

t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:", 
      t1_stop-t1_start)  

Elapsed time: 104.900577856 7.347182668
Elapsed time during the whole program in seconds: 97.553395188


In [28]:
track_search

('Clancy Lowered the Boom', 100, 1)

In [29]:
def song_recomender(name_song, n_songs):
  """returns indices from the model,
  pass them to df.loc[[]]"""
  id_track = process.extractOne(name_song,spotify['name'])[2]
  print('Song selected', spotify['name'][id_track], 'id:', id_track)
  print('searching for recomendation....')
  distances, ids = model_knn.kneighbors(spotify_matrix[[id_track]], n_neighbors=n_songs)
  # for i in ids:
  #   print(spotify[['name','artists','id']].loc[i])
  return ids

In [30]:
song = input('type name of the song and press Enter in your keyboard:')
print("will take a few seconds")
recomendations_20 = list(song_recomender(song,20)[0])
print(recomendations_20, type(recomendations_20))

will take a few seconds
Song selected Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve id: 0
searching for recomendation....
[0, 39, 109097, 39041, 92624, 20423, 1633, 1677, 1665, 75462, 40265, 39080, 41485, 109035, 125001, 38905, 75528, 156064, 126084, 108990] <class 'list'>


In [31]:
spotify.loc[recomendations_20]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,0.000000,0.985944,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.282389,0.153112,0.2110,0.0,4BJqT0PrAfrxzMOxytFOIz,0.878,0.909091,0.665,0.624916,1.0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.04,1921,0.037732,0.332450
39,0.0594,0.000000,0.985944,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.282389,0.153112,0.2110,0.0,1SCWBjhk5WmXPxhDduD3HM,0.878,0.909091,0.665,0.624916,1.0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.01,1921,0.037732,0.332450
109097,0.1020,0.070707,0.971888,"Sergei Rachmaninoff, Vladimir Horowitz, New Yo...",0.381579,0.164172,0.2100,0.0,7mqytyJkLxRlywbwSHFvic,0.921,0.909091,0.689,0.639903,1.0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.00,1928,0.037732,0.395225
39041,0.1020,0.070707,0.971888,"Sergei Rachmaninoff, Vladimir Horowitz, New Yo...",0.381579,0.164172,0.2100,0.0,2SPOwMPEXB4Jm1MKzDH8Wc,0.921,0.909091,0.689,0.639903,1.0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.00,1928,0.037732,0.395225
92624,0.1180,0.070707,0.997992,"Frédéric Chopin, Vladimir Horowitz",0.386640,0.084514,0.2050,0.0,6JFq3aShxlO5CyF2FrYXCr,0.907,1.000000,0.696,0.672853,1.0,"Scherzo No. 1 in B Minor, Op. 20",0.00,1928,0.041959,0.434411
20423,0.0859,0.070707,0.997992,"Frédéric Chopin, Vladimir Horowitz",0.247976,0.043672,0.1030,0.0,0zzGHfLjR6rcQhF0Oo1k8i,0.921,1.000000,0.672,0.645478,1.0,"Nocturne in E Minor, Op. 72, No. 1",0.02,1928,0.032887,0.363542
1633,0.0379,0.090909,0.985944,"Franz Liszt, Vladimir Horowitz",0.298583,0.145744,0.0713,0.0,34y6Gie19NTeapGMoqCHhl,0.891,1.000000,0.699,0.616256,1.0,"Années de pèlerinage, Première année (Suisse),...",0.14,1930,0.048660,0.309244
1677,0.0798,0.090909,0.936747,"Franz Liszt, Vladimir Horowitz",0.291498,0.113100,0.1450,0.0,7paJusVgjTQGvFZpTDHYBW,0.760,1.000000,0.717,0.635643,1.0,"Sonata for Piano in B Minor, S. 178: III. Alle...",0.09,1930,0.037010,0.435721
1665,0.0959,0.090909,0.992972,"Franz Liszt, Vladimir Horowitz",0.383603,0.081691,0.0877,0.0,3UaOfi16cKYbWt4EcHQJJr,0.868,0.818182,0.702,0.555759,1.0,"Soirées de Vienne, S. 427 No. 6: Allegro con s...",0.10,1930,0.047216,0.366322
75462,0.1280,0.070707,0.996988,"Frédéric Chopin, Vladimir Horowitz",0.302632,0.052736,0.0441,0.0,4gCfUCZsjnLU6q4RAAu3Oy,0.893,1.000000,0.670,0.550967,1.0,Andante,0.00,1928,0.038660,0.373706


##Save and pickle

In [34]:

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import files
else:
  print("No running in colab")


No running in colab


In [43]:
filename = './csv/spotify_track_neighbors.urenaj'
pickle.dump(model_knn, open(filename, 'wb'))
#pickle sklearn model

In [ ]:
#How to un-pickle 
# knn_2nd = pickle.load(open(filename, 'rb'))
# distances, x_ids = knn_2nd.kneighbors(spotify_matrix[[0]],3,return_distance=True)

In [40]:
X = spotify_matrix
np.savetxt("./csv/spotify_matrix.csv", X, delimiter=",")
#how to save or pickle np.array

In [ ]:
#load np.array 
# np.loadtxt("spotify_matrix.csv",delimiter=",")

In [42]:
spotify.to_csv('./csv/spotify_tracks.csv',index=False)